## Freni-Sterrantino et al 2017 - BYM2 connected, disconnected for Scotland Lip Cancer Dataset

Mitzi Morris

In [A note on intrinsic Conditional Autoregressive models for disconnected graphs](https://arxiv.org/abs/1705.04854), Freni-Sterrantino et.al. show how to implement the BYM2 model for use with areal data where the graph structure of the map is not fully connected.  In this notebook, we present that Stan implementation of this proposal.

The BYM2 model provides an intuitive parameterization for a GLM which has both an ICAR component $\phi$ which accounts for the spatial structure of the data, and a ordinaty random effects component $\theta$ for non-spatial heterogeneity.
In addition, the BYM2 model has a single precision (scale) parameter $\sigma$ on the combined components
and a mixing parameter $\rho$ for the amount of spatial/non-spatial variation.
In order for $\sigma$ to legitimately be the standard deviation of the combined components,
it is critical that for each $i$, $\operatorname{Var}(\phi_i) \approx \operatorname{Var}(\theta_i) \approx 1$.
This is done by adding a scaling factor $\tau$ to the model which scales 
the proportion of variance $\rho$.
Riebler et al. recommend scaling the model so the geometric mean of these variances is 1.
Because the scaling factor depends on the dataset, it comes into the model as data.

The Stan case study [Spatial Models in Stan: Intrinsic Auto-Regressive Models for Areal Data](https://mc-stan.org/users/documentation/case-studies/icar_stan.html) for details on the ICAR, BYM, and BYM2 models. 

### Overview of BYM2 model for a fully connected spatial structure

When the areal map is a single, fully connected component, i.e., a graph where any node in the graph can be reached from any other node, the BYM2 model is implemented as follows.

The spatial structure and scaling factor are data inputs to the model:

```
data {
  int<lower = 0> I;  // number of nodes
  int<lower = 0> J;  // number of edges
  int<lower = 1, upper = I> edges[2, J];  // node[1, j] adjacent to node[2, j]
  real tau; // scaling factor
```

The spatial and heterogeneous effects, combined variance, and proportion of spatial variance are model parameters:

```
parameters {
  real<lower=0, upper=1> rho; // proportion of spatial effect that's spatially smoothed
  real<lower = 0> sigma;  // scale of spatial effects
  vector[I] theta;  // standardized heterogeneous spatial effects
  vector[I] phi;  // standardized spatially smoothed spatial effects
```

The combined BYM2 component is computed in the `transformed parameters` block:

```
transformed parameters {
  // spatial effects (combine heterogeneous and spatially smoothed)
  vector[I] gamma = (sqrt(1 - rho) * theta + sqrt(rho / tau) * phi) * sigma;
```

The ICAR component is implemented as log probability density function which computes the ICAR pairwise difference and imposes a soft sum-to-zero constraint:

```
real standard_icar_lpdf(vector phi, int[ , ] adjacency) {
    return 0.5 * dot_self(phi[adjacency[1,]] - phi[adjacency[2]])
	  + normal_lpdf(sum(phi) | 0, 0.001 * rows(phi));
}
```

### Freni-Sterrantino recommendations for a disconnected graph and Stan implementation

Freni-Sterrantino et al show how to adjust the scaling factors when the areal map is not fully connected but has at least one connected multi-node component.  

1. Each connected component of size > 1 is scaled independently
2. Components of size 1 are scaled with a normal with precision K, where K is the number of components.

To extend the BYM2 model to these areal maps, we agument this model with a series of per-component masks into the node and edgelists and use Stan's multi-index operator and vectorized operations for efficient computation.

The spatial structure includes a set of arrays describing component-wise node, edgesets.
The `_cts` arrays record the size of the node and edgelists for each component, the `_idx` arrays provide the indices of the members of each component.

```
 int<lower=0, upper=I> K;  // number of components in spatial graph
  int<lower=0, upper=I> K_node_cts[K];   // per-component nodes
  int<lower=0, upper=J> K_edge_cts[K];   // per-component edges
  int<lower=0, upper=I> K_node_idxs[K, I];  // rows contain per-component node indexes
  int<lower=0, upper=J> K_edge_idxs[K, J];  // rows contain per-component edge indexes

  vector[K] tau; // scaling factor
```

Per recommendataion 1, above, the combined spatial and random effects in the BYM2 model are computed component-wise, each with their own scaling factor.  For singletons, the scaling factor `tau` is 1/K.

```
transformed parameters {
  vector[I] gamma;
  // each component scaled by tau[k]
  for (k in 1:K)
    gamma[K_node_idxs[k, 1:K_node_cts[k]]] = 
            (sqrt(1 - rho) * theta[K_node_idxs[k, 1:K_node_cts[k]]]
             +
             sqrt(rho / tau) * phi[K_node_idxs[k, 1:K_node_cts[k]]])
            * sigma;
```

The ICAR `_lpdf` function puts a sum-to-zero constraint on each component.
Singletons have normal spatial variance.

```
real standard_icar_disconnected_lpdf(vector phi,
				       int[ , ] adjacency,
				       int[ ] node_cts,
				       int[ ] edge_cts,
				       int[ , ] node_idxs,
				       int[ , ] edge_idxs) {
    real total = 0;
    for (n in 1:size(node_cts)) {
      if (node_cts[n] > 1)
        total += -0.5 * dot_self(phi[adjacency[1, edge_idxs[n, 1:edge_cts[n]]]] -
                                 phi[adjacency[2, edge_idxs[n, 1:edge_cts[n]]]])
                  + normal_lpdf(sum(phi[node_idxs[n, 1:node_cts[n]]]) |
                                      0, 0.001 * node_cts[n]);
      else
          total += normal_lpdf(phi[n] | 0, 1);  // iid spatial variance
    }
    return total;
}
```


### Python packages used in this notebook, including CmdStanPy

In [ ]:
import json

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline

import numpy as np
import pandas as pd

from cmdstanpy import cmdstan_path, CmdStanModel, install_cmdstan
# install_cmdstan()  # as needed - will install latest release (as needed)

### Areal data:  the counties in Scotland, circa 1980

The canonical dataset used to test and compare different parameterizations of ICAR models is a study on the incidence of lip cancer in Scotland in the 1970s and 1980s.  The data, including the names and coordinates for the counties of Scotland are available from R package [SpatialEpi](https://cran.r-project.org/web/packages/SpatialEpi/SpatialEpi.pdf), dataset `scotland`.

3 of these counties are islands:  the Outer Hebrides (western.isles), Shetland, and Orkney.  In the canonical datasets, these islands are conntected to the mainland, so that the adjacency graph consists of a single, fully connected component.  However, different maps are possible:  a map with 4 components, the mainland and the 3 islands; or a map with 3 components:  the mainland, a component consisting of Shetland and Orkney, and a singleton consisting of the Hebrides. The following plots demonstrate the differences:

In [ ]:
# figure size in inches optional
rcParams['figure.figsize'] = 11 ,8
img_A = mpimg.imread('scot_connected.png')
img_B = mpimg.imread('scot_3_comp.png')
img_C = mpimg.imread('scot_islands.png')
# display images
fig, ax = plt.subplots(1,3)
ax[0].imshow(img_A);
ax[1].imshow(img_B);
ax[2].imshow(img_C);

The graph nodes which have different amounds of connectivity are:  1: "sky-lochalsh" , 3: "caithnes", 5: "ross-cromarty", 6: "shetland", 8: "orkney", 9: "moray", 11: "western.isles", 12: "sutherland".

### Data prep:  from spatial polygon to 2D array of edges

In the Stan implementation of the ICAR model, the edgelist is a 2D array of size 2 $\times$ J where J is the number of edges in the graph.  Each column entry in this array represents one undirected edge in the graph, where for each edge j, entries [j,1] and [j,2] index the nodes connected by that edge.  Treating these are parallel arrays and using Stan's vectorized operations provides a transparent implementation of the pairwise difference formula used to compute the ICAR component.

The common format for the spatial structure of an areal dataset is as a set of shapefiles.  The areal regions are described by a set of spatial polygons, i.e., a description of the shape of each region in terms of its lat,lon coordinates.  The R package [spdep](https://r-spatial.github.io/spdep/index.html) extracts the adjacency relations as a `nb` object.
We have written a set of helper functions which take the `nb` objects for each graph into the set of data structures needed by the Stan models, these are in file `bym2_helpers.R`.  The helper function `nb_to_edge_array` takes the `nb` object and returns the 2 $\times$ J edge array; the helper function `scaling_factor` uses the edge array to compute the geometric mean of the corresponding adjacency matrix.


The `scotland` dataset contains the shapefiles for the counties in Scotland.  The fully connected graph corresponds to the data as distributed.  By editing the `nb` objects, we have created the alternative maps above.
The three versions of the Scotland spatial structure are in files `scotland_nbs.data.R`, `scotland_3_comp_nbs.data.R`, and `scotland_islands_nbs.data.R`.
The file `munge_scotland.R` munges the data, and it has been saved as JSON data files.

### Regression data:  the Scotland cancer dataset

The cancer study data is:

- `y`: observed outcome - number of cases of lip cancer
- `x`: single predictor - percent of population working in agriculture, forestry, or fisheries.
- `E`: population
    
This dataset is available via several different R packages; often `x` is called `AFF`; sometime it is given as a percentage, i.e., scaled from 1 to 100; sometimes as a proportion, scaled from 0 to 1.  The version of this dataset distributed as a BUGS example scales the percentage by 1/10, as does the INLA model.  In order to compare the Stan results with INLA et al, in this dataset, `x`, the percenatage population, is also scaled by 1/10.

### Fitting the BYM2_islands model to Scotland map as mainland component plus islands

For the Scotland map with 3 island (singleton) components, in file `scotland_islands_nbs.data.R`, we use function index_components and write_json produce the input data file `scotland_islands.data.json`.   This map has the same nodes set as in the `scotland` dataset, but the neighbors object (file `scotland_nbs.data.R`) has been edited to remove all edges between islands and the mainland or each other.

In [ ]:
with open('scotland_islands.data.json') as fd:
    islands_data = json.load(fd)

print('num nodes: {}, num edges: {}'.format(islands_data['I'], islands_data['J']))

The `index_components` function creates the per-component node and edge index arrays, and computes the vector of scaling factors `tau`:

In [ ]:
print('num components: {}\nscaling factors: {}\nnodes per component: {}\nnode indices: {}'.format(
    islands_data['K'], islands_data['tau'], islands_data['K_node_cts'], islands_data['K_node_idxs']))

We use CmdStanPy to compile and fit the model using 

In [ ]:
from cmdstanpy import cmdstan_path, CmdStanModel
bym2_islands_model = CmdStanModel(stan_file='bym2_islands.stan')

In [ ]:
print(bym2_islands_model.code())

We fit the model using the data dictionary `islands_data`, alternatively, one can specify file `scotland_islands.data.json`.  For reproducibility, we specify the seed supplied to the Stan pseudo-random number generator used by the sampler.

In [ ]:
bym2_islands_fit = bym2_islands_model.sample(data=islands_data, seed=12345)

The CmdStanMCMC method `summary` wraps the CmdStan [`stansummary`](https://mc-stan.org/docs/cmdstan-guide/stansummary.html) utility; it returns a pandas DataFrame object, one row per summary output row for the joint log probability density `lp__` and for the values of all variables in the Stan program.

In [ ]:
islands_summary = bym2_islands_fit.summary()
islands_summary

The R_hat and N_Eff (number of effective samples) in the summary report indicate that the model has fit the data.  To further check the fit, we run the `diagnose` method, which wraps CmdStan's [`diagnose`](https://mc-stan.org/docs/cmdstan-guide/diagnose.html) utility.

In [ ]:
bym2_islands_fit.diagnose()

The diagnose command detects _potential_ problems by examining the set of sampler diagnostic variables; these are the initial columns of the Stan CSV output file, which have column labels which end in `__`.  For example, to further check the treedepth, which is an integer output between 1 and sampler configuration argument `max_treedepth`, default 10, we can access this column from the output and do a quick summary of the per-draw treedepth counts:

In [ ]:
bym2_islands_draws = bym2_islands_fit.draws(concat_chains=True)
treedepth_idx = bym2_islands_fit.column_names.index('treedepth__')
treedepths = np.bincount(bym2_islands_draws[:, treedepth_idx].astype('int'))
for idx, val in enumerate(treedepths):
    if val > 0:
        print(idx, val)

To get the summary statistics for a Stan program variable across all chains, we can access individual rows of the pandas DataFrame object using [`iloc`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html).  Here we examine the estimates for the quantities `theta` and `phi` for the first 12 nodes in the graph:

In [ ]:
names = list(islands_summary.index)
theta_rows = [names.index(name) for name in names if name.startswith('theta[')]
phi_rows = [names.index(name) for name in names if name.startswith('phi[')]
print('spatial effects:\n{}\n\nheterogenous effects\n:{}'.format(
    islands_summary.iloc[phi_rows,:][0:11], 
    islands_summary.iloc[theta_rows,:][0:11]))


## Fit connected graph on Scotland Lip cancer dataset with BYM2 model implemented in Stan.

In [ ]:
from cmdstanpy import cmdstan_path, CmdStanModel, install_cmdstan
# install_cmdstan()  # as needed - will install latest release (as needed)

The dataset `scot_connected.data.json` contains the cancer dataset together with the spatial structure.
The spatial structure is comprised of:

- I: `int<lower = 0> I;  // number of nodes`
- J: `int<lower = 0> J;  // number of edges`
- edges: `int<lower = 1, upper = I> edges[2, J];  // node[1, j] adjacent to node[2, j]`
- tau: `real tau; // scaling factor`

In [ ]:
with open('scotland_connected.data.json') as fd:
    connected_data = json.load(fd)

bym2_model = CmdStanModel(stan_file='bym2.stan')
bym2_fit = bym2_model.sample(data=connected_data, seed=12345)

In [ ]:
connected_summary = bym2_fit.summary()
connected_summary

We compare the different estimates for model parameters `alpha`, `beta`, `rho`, and `sigma` for the connected and disconnected fits:

In [ ]:
print('connected\n{}\n\nislands\n{}'.format(connected_summary.iloc[1:5,:], islands_summary.iloc[1:5,:]))

For the connected and islands graphs, the nodes which have different connectivity are: (1,3,5,6,9,11,12).  To see how this affects the estimates, we look at the differences between parameters `phi` and `theta`.

In [ ]:
print('connected\n{}\n\nislands\n{}'.format(connected_summary.iloc[phi_rows,:][0:14], islands_summary.iloc[phi_rows,:][0:14]))

In [ ]:
phi_mean_connected = connected_summary.iloc[phi_rows,0]
phi_mean_islands = islands_summary.iloc[phi_rows,0]
plt.scatter(phi_mean_connected, phi_mean_islands)

In [ ]:
print('spatial effects:\n{}\n\nheterogenous effects\n:{}'.format(
    connected_summary.iloc[phi_rows,:][0:11], 
    connected_summary.iloc[theta_rows,:][0:11]))

We can also fit the connected graph as if it were a disconnected graph; the `BYM2_islands.stan` program allows graphs with a single component as input, all we need to do is supply the additional input data structures "K", "K_node_cts", etc.

In [ ]:
with open('scotland_connected_as_Kgraph.data.json') as fd:
    s2_data = json.load(fd)
bym2_islands_model = CmdStanModel(stan_file='bym2_islands.stan')
bym2_s2_fit = bym2_islands_model.sample(data=s2_data, seed=12345)

s2_summary = bym2_s2_fit.summary()

Both models returns the same estimates, although the islands model is slower, due to the overhead of the indexing operations.

In [ ]:
print('connected, fit with bym2_model\n{}\n\nconnected, fit with bym2_islands model\n{}'.format(connected_summary.iloc[1:5,:], s2_summary.iloc[1:5,:]))

### Mainland plus 1 singleton, and combined Shetland + Orkney component

For the Scotland map with 3 components, in file `scotland_3_comps_nbs.data.R`, the islands of Shetland and Orkney have been combined.  This dataset exists to test additional possible kinds of maps, but provides no additional insights on the model or the domain data.

In [ ]:
bym2_c3_fit = bym2_islands_model.sample(data='scotland_3_comp.data.json', seed=12345)

In [ ]:
c3_summary = bym2_c3_fit.summary()
c3_summary